# Projenin başlığı

#### Öğrenci ad(lar)ı

Abstract of the project comes here.

## İçerik

[Problem](#problem)   
[Verinin Anlaşılması](#data_understanding)   
[Verilerin Hazırlanması](#data_preparation)   
[Modelleme](#modeling)   
[Değerlendirme](#evaluation)   
[Referanslar](#references)   


## Problem <a class="anchor" id="problem"></a>

Describe the problem here. What are the questions you are trying to solve?

## Verinin Anlaşılması<a class="anchor" id="data_understanding"></a>

What data set are you going to use? Where will you get it from? Describe your data.

## Verilerin Hazırlanması <a class="anchor" id="data_preparation"></a>

Explain what kind of data transformations, feature selection and/or engineering you will perform.

## Modelleme <a class="anchor" id="modeling"></a>

Which model will be used? Why? What parameters?

## Değerlendirme <a class="anchor" id="evaluation"></a>

Evaluate your model. Provide results, tables, charts, etc.

## Referanslar<a class="anchor" id="references"></a>

List all the sources you used during your work.
This includes jupyter notebooks you found on Internet.
Remeber, your work may not be an original one. However, this document must be genuine. Copy and paste kind of deliveries will be punished badly.


**Disclaimer!** <font color='grey'>This notebook was prepared by <student name(s)> as a term project for the *BBM469 - Data Intensive Applications Laboratory* class. The notebook is available for educational purposes only. There is no guarantee on the correctness of the content provided as it is a student work.

If you think there is any copyright violation, please let us [know](https://forms.gle/BNNRB2kR8ZHVEREq8). 
</font>